- `songs_preprocessed = [roll]`. El equivalente actual es `rolls_preprocessed = [r.matrix for r in df['roll']]`

- `roll = ndarray(16*n,89)` donde `n` es la cantidad de compases

Modelo Bach-ragtimes de 4 compases tardó 1000 épocas

In [ ]:
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')

    # Change the directory
    os.chdir('/content/drive/MyDrive/ColabNotebooks/')
    is_colab = True
    print("Running on Colab")

except:
    is_colab = False
    print("Running locally")

dataset_path = './data/'
path_saved_models = './saved_models'

In [ ]:
#@title ¿Entrenamos un nuevo modelo? - Seleccionar directorios de los datasets
if_train = "Si" #@param ["Si", "No"]
entrenar_nuevo = "Si" #@param ["Si", "No"]

if entrenar_nuevo:
  epoca_inicial =  0#@param {type:"integer"}

epoca_final =  1#@param {type:"integer"}
checkpt =  1#@param {type:"integer"}

dataset1 = "Bach/" #@param {type:"string"}
dataset2 = "ragtime/" #@param {type:"string"}
dataset3 = "Mozart/" #@param {type:"string"}
dataset4 = "Frescobaldi/" #@param {type:"string"}

In [ ]:
#@title Transformar estilo
ds_original = "Bach" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]
ds_objetivo = "Mozart" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]

nombre_pickle = ds_original+"2"+ds_objetivo

#### Setup

##### Bibliotecas y constantes

In [ ]:
# 35s
if is_colab:
    ! pip install pretty_midi
    ! apt-get install musescore
    ! sudo apt install lilypond
    ! pip install dfply

    ! pip install pyfluidsynth
    ! apt install musescore fluidsynth

    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [ ]:
#3s
import random
random.seed(42)

In [ ]:
from utils.files_utils  import save_pickle, load_pickle, datasets_name
from utils.plots_utils import plot_train, calculate_TSNEs, plot_tsne
from utils.utils import exp_disponibles, filter_column
from display_audio import save_audios, display_audios, display_score
from roll.guoroll import get_scores_from_roll

____
# Music 21
### No hace falta correrlo para entrenar, salvo que sea algo nuevo

31s

## Preprocessing: Estandarización del dataset


Se lo lleva al formato lista de:

- $n$ canciones = lista de:
- 2 voces = lista de:
- $e$ eventos (notas, silencios, etc.)

In [ ]:
songs = {
    dataset1[:-1]: [dataset1+path for path in os.listdir(dataset_path+dataset1)],
    dataset2[:-1]: [dataset2+path for path in os.listdir(dataset_path+dataset2)],
    dataset3[:-1]: [dataset3+path for path in os.listdir(dataset_path+dataset3)],
    dataset4[:-1]: [dataset4+path for path in os.listdir(dataset_path+dataset4)],
}

### Dataset con DataFrame


Armamos el DataFrame con el que manejamos las canciones. Cada fila es una canción y las columnas son:
- `Autor`
- `Título`
- `Id roll`: qué matriz es de la canción
- `Roll` matriz de $16*n \times 89$ con $n=$ la cantidad de compases
- `Old PM`: Pretty MIDI original

Luego le agregaremos:
- `Embedding`: el vector resultado de encodear el roll

In [ ]:
from preprocessing import preprocess_data

df_midi = preprocess_data(songs)

In [ ]:
save_pickle(df=df_midi, name='ragtime_mozart_df')

In [ ]:
# (df >> dfply.group_by('Autor')
#   >> dfply.summarise(count=dfply.X['Titulo'].unique().shape[0]))

In [ ]:
# (df >> dfply.group_by('Autor', 'Titulo')
#   >> dfply.summarise(count=dfply.X['Id roll'].shape[0])
#   >> dfply.group_by('Autor')
#   >> dfply.summarise(count_autor=dfply.X['count'].mean())
#   )

# Modelo de GUO 


### Modelo (entrenamiento o carga)

Para ver la imagen de la partitura:
- `lily = lily_conv.write(`**stream**`, fmt='lilypond', fp='file', subformats=['png'])`
- `Image(str(lily))`

In [ ]:
from model.train import continue_training

# rolls_preprocessed = np.stack([r.matrix for r in df_midi['roll']])
# vae, callbacks = continue_training(rolls_preprocessed, if_train, entrenar_nuevo, epoca_final)
vae, callbacks = continue_training(df_midi, if_train, entrenar_nuevo, epoca_final)

In [ ]:
plot_train(callbacks, epoca_final)

___
## Embeddings

#### Cálculo de embeddings

In [ ]:
from model.embeddings import obtain_embeddings

df_emb = obtain_embeddings(df_midi, vae)
df_emb

### Vectores característicos

In [ ]:
from model.embeddings import obtain_characteristics

df_emb_car, df_caracteristicos, caracteristicos_de_autores = obtain_characteristics(df_emb)

caracteristicos_pkl_name = 'df_car'+datasets_name(songs)
save_pickle(df_caracteristicos, caracteristicos_pkl_name)
save_pickle(caracteristicos_de_autores, 'caracteristicos_de_autores')

### Creación de rolls y midis

In [ ]:
decoder = vae.get_layer(name='decoder')

## Transferir estilos

In [ ]:
from model.embeddings import cambiar_estilo, get_embeddings_roll_df

#@title ¿Ya reconstruye? ¿Transferimos estilos o solo probamos la reconstrucción?
transfer = False #@param {type:"boolean"}

df_transfered = cambiar_estilo(df_emb_car, caracteristicos_de_autores, ds_original, ds_objetivo, escala=1) if transfer else df_emb_car
# save_pickle(df_transfered, nombre_pickle)


# TODO: Llegué hasta acá refactorizando
exp_list = [e for e in exp_disponibles(df_transfered) if 'roll' not in e]
print("Lista de experimentos:")
print(exp_list)

for exp in exp_list:
  get_embeddings_roll_df(df_transfered, vae, column=exp, inline=True)

save_pickle(df_transfered, nombre_pickle)
print("Guardado en " + nombre_pickle + ".pkl")

___
# Working space
Por default, usar `ragtime_mozart_df`. Tiene:
- Rolls
- pretty_midis
Entonces no hace falta correr Music 21.

In [ ]:
# current_df = df_midi
current_df = load_pickle('ragtime_mozart_df')
current_df.columns

In [ ]:
save_pickle(current_df['roll'][0].matrix.shape, 'roll_ejemplo')
sonata545_df = load_pickle('sonata545')

In [ ]:
# canciones_a_imprimir['stoptime.mid']
df_emb = load_pickle('df_emb_car')

# list(df_emb[df_emb['Autor']=='Mozart']['Titulo'])

df_moz = df_emb[df_emb['Autor']=='Mozart']
df_moz[df_moz['Titulo'].str.contains('sonata15')].head(2)

save_pickle(df_moz[df_moz['Titulo'].str.contains('sonata15')].head(2), "df_head")

dict_df = {}

for nombre in current_df['Autor']:
  cantidad = len(list(current_df[(current_df['Autor'] == nombre) & (current_df['Tipo'] == 'Titulo')]))
  dict_df[nombre] = cantidad
  # print(nombre, cantidad)

dict_df

In [ ]:
df_completo = get_embeddings_roll_df(df_emb, vae)

In [ ]:
# TODO: Celda probablemente deprecada
from roll.guoroll import _roll_to_midi

current_df['Roll midi'] = _roll_to_midi(current_df['roll'].matrix, current_df['Old PM'])
# TODO: no va a funcionar la línea de arriba. Antes era roll_to_midi(current_df['Roll'], ...)current_df.columns

In [ ]:
callbacks.history.items()

___
# Visualizaciones

## Gráficos
Visualización de embeddings

In [ ]:
tsne_emb = calculate_TSNEs(current_df)

# plot_tsnes_comparison(df_emb, tsne_emb[1:])
plot_tsne(current_df, tsne_emb[0])

## MIDIs

Tomamos las matrices generadas y las volvemos a convertir en midis

In [ ]:
def emb_disponibles(df):
  return [c[:-5] for c in df.columns if "midi" in c]

#@title # Escuchar MIDIs
pickle_df = "ragtime2Mozart" #@param ["ragtime2bach", "bach2ragtime", "Bach2Mozart", "Mozart2Bach", "Mozart2Frescobaldi", "Mozart2ragtime", "ragtime2Mozart", "Frescobaldi2Mozart", "Frescobaldi2Bach", "ragtime2Frescobaldi"]
current_df = load_pickle(pickle_df)

# @ markdown Opciones a escuchar:

# for e in emb_disponibles(current_df):
#   print(e)

In [ ]:
columna = "Embedding" #@param ["Roll", "Embedding", "Mutacion_add", "Mutacion_add_sub"]
muestras = 2 #@param {type:"slider", min:0, max:10, step:1}

midis = list(zip(current_df['Titulo'], current_df['Id roll'], current_df[columna+' midi'], current_df['Roll midi']))[:muestras]

# #@markdown # ¿Eliminar midis viejos?
# eliminar = True #@param {type:"boolean"}
# if eliminar:
#   !rm -rf /content/drive/MyDrive/ColabNotebooks/MIDIs_Generados/Guo/*
# cancion = input("Nombre de los archivos nuevos:")

save_audios(midis)
display_audios(midis)

In [ ]:
columna = "Embedding" #@param ["Roll", "Embedding", "Mutacion_add", "Mutacion_add_sub"]
muestras = 5 #@param {type:"slider", min:0, max:10, step:1}

midis = list(zip(current_df['Titulo'], current_df['Id roll'], current_df[columna+' midis'], current_df['Roll midis']))[:muestras]

save_audios(midis)
display_audios(midis)

In [ ]:
columna = "Mutacion_add_sub" #@param ["Roll", "Embedding", "Mutacion_add", "Mutacion_add_sub"]
muestras = 5 #@param {type:"slider", min:0, max:10, step:1}

midis = list(zip(current_df['Titulo'], current_df['Id roll'], current_df[columna+' midis'], current_df['Roll midis']))[:muestras]

save_audios(midis)
display_audios(midis)

## Partituras (falta actualizar; hoy no funciona)
Asignar a `canciones_a_imprimir` el diccionario `{nombre: roll}`

#### Imprimir

In [ ]:
# [nombre for nombre, roll in canciones_a_imprimir]

canciones_a_imprimir = filter_column(current_df, 'roll')
embeddings_a_imprimir = filter_column(current_df, 'Embedding')

In [ ]:
# def get_scores_from_roll(roll):
#   return roll_to_score(roll.T)

# noised_scores = get_scores_from_roll(noised_roll)
# dup_scores = get_scores_from_roll(dup_roll)
# sigma_scores = get_scores_from_roll(sigma_roll)
# progression_scores = get_scores_from_roll(dup_progression_roll)

# all_scores = [noised_scores, dup_scores, sigma_scores, progression_scores]
scores =  { nombre: get_scores_from_roll(roll)
    for nombre, roll in canciones_a_imprimir.items()}

In [ ]:
# lily = lily_conv.write(original_score, fmt='lilypond', fp='file', subformats=['png'])
# display(Image(str(lily)))

In [ ]:
for nombre, score in scores.items():
  print(nombre)
  for i,s in enumerate(score):
    print(i)
    display_score(s)